# INFERENCE EXAMPLE NOTEBOOK
This notebook demonstrates how the inference pipeline can be used. We first create the inference pipeline, which pulls all necessary items from the public hosting in S3, then we demonstrate the required functionalities:
1. Team ranking
2. Tournament ranking
3. Global ranking 

For each functionality, we return two objects
1. The ranking 
2. The explanations - these show the top 5 positive and negative factors for each team, based on the specific matchups they play against in the team/tournament/global set. 

In [1]:
import os
from data_processing.InferenceProcessor import InferencePipeline

# Change to the directory where you want data to be stored
os.chdir('esports-data-aws')
# Creating the inference pipeline object pulls objects from S3 and assembles them to be ready for inference
inference_fn = InferencePipeline()

Loading data from S3...
Loading metadata files...
Loading game data...
Loading model...


In [4]:
# DEMONSTRATE TEAM RANKING - we get the IDs of all teams at Worlds 2023 and compute their rankings
worlds_teams = ['Gen.G', 'T1', 'KT Rolster', 'Dplus', 'Cloud9', 'NRG', 'Team Liquid', 'WeiboGaming FAW Audi', 'Beijing JDG Intel', 'bilibili', 'suzhou LNG',
    'g2 esports', 'fnatic', 'MAD lions', 'golden guardians', 'team BDS', 'LOUD', 'GAM esports', 'TEAM WHALES', 'psg talon', 'ctbc flying oyster', 'detonation focusme', 
    'Movistar R7']

# This function does an approximate text match to find the team ID for a given team name, it also prints out the matched text to help check that you got the team you intended
worlds_team_ids = [inference_fn.find_team_id(team) for team in worlds_teams]

Gen.G
T1
kt Rolster
Dplus Kia
Cloud9
NRG
Team Liquid Honda
WeiboGaming FAW AUDI
Beijing JDG Intel Esports Club
Bilibili Gaming Pingan Bank
Suzhou LNG Esports
G2 Esports
Fnatic
MAD Lions
Golden Guardians
Team BDS
LOUD
GAM Esports
TEAM WHALES
PSG Talon
CTBC Flying Oyster
DetonatioN FocusMe
Movistar R7


In [5]:
# Now run inference to get the rankings and explanations
team_ranks, team_top_features = inference_fn.get_team_rankings(worlds_team_ids)
team_ranks 

,team_name,score
0,Gen.G,11.226745
1,Beijing JDG Intel Esports Club,11.224624
2,kt Rolster,11.199479
3,WeiboGaming FAW AUDI,10.193123
4,Bilibili Gaming Pingan Bank,9.164375
5,Dplus Kia,8.112445
6,Suzhou LNG Esports,7.145285
7,G2 Esports,6.165362
8,T1,6.108284
9,MAD Lions,3.127975


In [10]:
# Next we get the feature importances. This will give us the top 5 positive factors and top 5 negative factors for each team (we shorten to 3 for presentation purposes)
# In short, this is done by computing the most important features on average for each match that a team plays. See methodology document for more details.

# Show the positive values first - these are factors that cause the model to predict a higher win probability for the team
team_top_features[['top_1_pos', 'top_2_pos', 'top_3_pos']].head(5)

,top_1_pos,top_2_pos,top_3_pos
T1,Diff in mid damage/min @ 7,Diff in support damage/min to buildings advant...,Diff in top XP/min advantage @ 20
Fnatic,Diff in mid building damage @ 20,Diff in mid gold spent @ 20,Diff in mid damage/min advantage @ 20
Cloud9,Diff in bot XP/min advantage @ game end,Diff in bot XP/min @ game end,Diff in mid gold spent @ 20
G2 Esports,Diff in bot XP/min advantage @ game end,Diff in mid building damage @ 20,Diff in mid gold/min advantage @ 20
Movistar R7,Diff in bot XP/min advantage @ game end,Diff in support damage to buildings advantage ...,Diff in mid lane damage to buildings advantage...


In [12]:
# Show the negative values - these are factors that cause the model to predict a loss
team_top_features[['top_1_neg', 'top_2_neg', 'top_3_neg']].head(5)

,top_1_neg,top_2_neg,top_3_neg
T1,Diff in bot XP/min advantage @ game end,Diff in mid gold/min advantage @ 20,Diff in bot gold/min advantage @ game end
Fnatic,Diff in bot XP/min advantage @ game end,Diff in bot gold/min advantage @ game end,Diff in mid KDA advantage @ 20
Cloud9,Diff in mid damage/min advantage @ 20,Diff in mid gold/min advantage @ 20,Diff in top damage/min @ 7
G2 Esports,Diff team CS adnvatage @ 30,Diff in bot XP/min @ game end,Diff in mid damage/min advantage @ 20
Movistar R7,Diff in mid building damage @ 20,Diff in mid damage/min advantage @ 20,Diff in mid KDA advantage @ 20


In [13]:
# DEMONSTRATE TOURNAMENT RANKINGS FUNCTIONALITY 
# Get rankings for LCS Spring 2023
team_ranks, team_top_features = inference_fn.get_tournament_rankings('109517090066605615', 'Regular Season') 

team_ranks.head(5)

,team_name,score
0,Evil Geniuses LG,4.539441
1,Team Liquid Honda,4.536010
2,Cloud9,4.528998
3,100 Thieves,4.521282
4,FlyQuest,4.515735


In [14]:
# DEMONSTRATE GLOBAL RANKINGS FUNCTIONALITY
team_ranks, team_top_features = inference_fn.get_global_rankings(k=20)

team_ranks.head(5)

,team_name,score
0,Beijing JDG Intel Esports Club,227.630401
1,Gen.G,227.574044
2,kt Rolster,227.562608
3,WeiboGaming FAW AUDI,225.974994
4,Bilibili Gaming Pingan Bank,224.494781
